In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [3]:
news_dataset = pd.read_csv('train.csv')
news_dataset1 = pd.read_csv('test.csv')

In [4]:
news_dataset.shape

(20800, 5)

In [5]:
#hien thi 5 dong dau tien
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
#so luong bi thieu trong dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
#thay the so luong null bang string rong
news_dataset = news_dataset.fillna('')
news_dataset1 = news_dataset1.fillna('')

In [8]:
#hop nhat ten tac gia va tieu de
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']
news_dataset1['content1'] = news_dataset1['author']+' '+news_dataset1['title']

In [9]:
print(news_dataset['content'])
#news_dataset.head()

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [10]:
#tach data va label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

print(X)
print('--------------------------')
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [11]:
port_stem = PorterStemmer() #đưa các từ về dạng nguyên mẫu

In [12]:
def stemming(agr):
    stemmed_content = re.sub('[^a-zA-z]',' ', agr)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [13]:
news_dataset['content'] = news_dataset['content'].apply(stemming)
news_dataset1['content1'] = news_dataset1['content1'].apply(stemming)

In [14]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [15]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

P = news_dataset1['content1'].values

In [16]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [17]:
print(Y)

[1 0 1 ... 0 1 1]


In [18]:
#converting du lieu van ban sang du lieu so
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
vectorizer.fit(P)


X = vectorizer.transform(X)
K_test = vectorizer.transform(P)
print(X)

  (0, 8440)	0.30637649390862737
  (0, 7210)	0.2909577136736758
  (0, 4629)	0.34214314503179805
  (0, 4167)	0.3121403148732912
  (0, 3804)	0.25509319068205105
  (0, 2691)	0.2772114975963559
  (0, 2052)	0.3210623876107006
  (0, 1571)	0.2715587684340332
  (0, 1304)	0.4337415027138284
  (0, 147)	0.3163982964753294
  (1, 8994)	0.31382147886721357
  (1, 3699)	0.1945215363335672
  (1, 2999)	0.7133615317876577
  (1, 1932)	0.2651920557432443
  (1, 1484)	0.19469666733924623
  (1, 1156)	0.36142956889835864
  (1, 979)	0.15437033055741373
  (1, 785)	0.303475905694002
  (2, 8399)	0.42489598472948564
  (2, 5157)	0.4951808375391314
  (2, 3264)	0.3594826818588411
  (2, 2935)	0.38263854579603196
  (2, 1669)	0.4478645702386819
  (2, 1559)	0.31310798733727463
  (3, 8582)	0.25621216601337693
  :	:
  (20797, 7035)	0.2479405960124089
  (20797, 6602)	0.28761420145284444
  (20797, 6492)	0.24609677725734458
  (20797, 5524)	0.07987188194883543
  (20797, 5144)	0.173601869788456
  (20797, 5109)	0.30876252310723
  

In [19]:
#tach tap du lieu thanh du lieu huan luyen va du luyen test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size =0.2, stratify=Y, random_state=2)

In [20]:
#su dung mo hinh huan luyen: Logistic Regression
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [21]:
#danh gia su chinh xac tren du lieu train

X_train_prediction = model.predict(X_train)
print(X_train_prediction)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

print('Su chinh xac tren du lieu train: ', training_data_accuracy)

[0 0 0 ... 0 0 1]
Su chinh xac tren du lieu train:  0.9855769230769231


In [22]:
#danh gia su chinh xac tren du lieu test
X_test_prediction = model.predict(X_test)
print(X_test_prediction)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

print('Su chinh xac tren du lieu test: ', test_data_accuracy)

[1 0 1 ... 1 1 0]
Su chinh xac tren du lieu test:  0.9798076923076923


In [23]:
#tao ra 1 he thong du doan



X_new = K_test[1]

prediction = model.predict(X_new)

#prediction1 = model.decision_function(X_new) #diem tin cay cho mau
#print(prediction1)

prediction2 = model.predict_proba(X_new) #xac suat roi vao
print(prediction2)



if (prediction[0] == 0):
    print('day la tin that')
else:
    print('day la tin gia')

[[0.08557646 0.91442354]]
day la tin gia


In [24]:
K_test_prediction = model.predict(K_test)
print(K_test_prediction)

[0 1 1 ... 0 1 0]
